In [ ]:
# ✅ 필수 라이브러리 설치
!pip install ultralytics --upgrade

In [ ]:
import gdown
import zipfile
import os

file_id = "1OBOYX9v_0IvNWoJDIT4nuEKuHyKhSZDS"
gdown.download(f"https://drive.google.com/uc?id={file_id}", quiet=False)

zip_path = "/content/cring_oring.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall()

print(f"압축 해제 완료")

In [ ]:
import gdown
import zipfile
import os

file_id = "1ZNAf_OXTpFR6pNE1DbResLAzrvKZVuJ6"
gdown.download(f"https://drive.google.com/uc?id={file_id}", quiet=False)

In [ ]:
import yaml

data = {
       "train" : './train',
        "val" : './val',
        "names" : {
                        "0":"bottom_normal",
                        "1":"bottom_c_n_o_empty",
                        "2":"bottom_c_empty",
                        "3":"bottom_o_empty",
                        "4":"top_normal"
                  }
    }

with open('./data.yaml', 'w') as f :
    yaml.dump(data, f)

# check written file
with open('./data.yaml', 'r') as f :
    lines = yaml.safe_load(f)
    print(lines)

## 🏋️ 전이 학습을 통한 사용자 정의 모델 학습

In [ ]:
# 전이 학습 시작
from ultralytics import YOLO
model = YOLO("yolo11n.pt")
model.train(data='data.yaml', epochs=20, imgsz=640, batch=8, name='defect-detection')

## 🔎 학습된 모델로 테스트

In [ ]:
# 모델 불러오기 및 예측
model = YOLO('/content/runs/detect/defect-detection/weights/best.pt')
results = model('/content/val/images/26_16-38-13_1.png', save=True, show=True)
results

In [ ]:
import cv2
import matplotlib.pyplot as plt

# 이미지 불러오기
img_path = "/content/val/images/26_16-38-13_1.png"
img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 결과 정보 얻기
boxes = results[0].boxes
annotated = img_rgb.copy()

# 시각화 (Matplotlib 사용)
for box in boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cls_id = int(box.cls[0])
    conf = float(box.conf[0])
    label = model.names[cls_id]

    # 박스 그리기
    cv2.rectangle(annotated, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # 라벨 그리기
    text = f"{label} {conf:.2f}"
    cv2.putText(annotated, text, (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

# 6. 출력
plt.figure(figsize=(10, 6))
plt.imshow(annotated)
plt.axis("off")
plt.title("YOLOv11 Prediction")
plt.show()

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline

plt.figure(figsize=(8,8))
index=0
target = "/content/val/images/*.png"
for i, image_name in enumerate(glob.glob(target)[index:index+40]):
    print(i, image_name)
    results = model(image_name)
    plots = results[0].plot(conf=True, line_width=2)
    plt.subplot(8,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.subplots_adjust( wspace=0, hspace=0)
    plt.imshow(plots)
plt.show()

In [ ]:
!pwd